In [1]:
from sklearn.datasets import make_blobs
from sklearn.decomposition import PCA
from lib.CF_De.cfde import CF_Descent
from lib.cf_a.cfae import CF_Ascent
from lib.eval.eval_metrics import *
import numpy as np
import random
from lib.eval.tools import center_prediction

%load_ext autoreload
%autoreload 2 

# Evaluation of custom algorithms specifications

In this workspace we test what parameters works best for CFDE and CFAE.

In [2]:
X, y, centers = make_blobs(centers=4, cluster_std=8, n_features=30, n_samples=1500, return_centers=True)

pca = PCA(n_components=2)
embed_X = pca.fit_transform(X)



## CFDE

## Immutable ratio

In [ ]:
values = np.linspace(0.0,1.0,num=31)

n = 10
target = 1.0
pred = 1.0
while pred == target:
    index = random.randint(0,len(X))
    new_instance = X[index]
    pred = y[index]

best = -1
best_metric = -float('inf')
for ratio in values:
    metric = 0
    for i in range(n):
        x, cf, h = CF_Descent(X,y,1.0,centers,instance_index=index,new_immutable_ratio=ratio)
        x = x.reshape(1,-1)
        cf = cf.reshape(1,-1)

        metric += cf_similarity(x, cf)[0] * cf_minimality(cf, x) * cf_plausibility(cf, target, X, y)[0]
    metric = metric / n
    if metric > best_metric:
        best_metric = metric
        best = ratio

print("Best immutable ratio is:", best)

Features that can be changed count:  29
Amount of changes:  496
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  537
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  422
Number of changed features: 10
Features that can be changed count:  29
Amount of changes:  391
Number of changed features: 10
Features that can be changed count:  29
Amount of changes:  478
Number of changed features: 11
Features that can be changed count:  29
Amount of changes:  531
Number of changed features: 13
Features that can be changed count:  29
Amount of changes:  495
Number of changed features: 11
Features that can be changed count:  29
Amount of changes:  501
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  495
Number of changed features: 12
Features that can be changed count:  29
Amount of changes:  514
Number of changed features: 13
Features that can be changed count:  28
Amount of 

KeyboardInterrupt: 

## CFAE

## CFDE vs CFAE

In [34]:
it = 10
n = 10
target = 1.0

count_sim_de = 0
count_sim_ae = 0

count_min_de = 0
count_min_ae = 0

count_plaus_de = 0
count_plaus_ae = 0

for _ in range(it):
    pred = 1.0
    while pred == target:
        index = random.randint(0,len(X))
        new_instance = X[index]
        pred = y[index]

    metric_sim = [0,0]
    metric_min = [0,0]
    metric_plaus = [0,0]
    for i in range(n):
        x1, cf1, h1 = CF_Descent(X=X,y=y,target=1.0,centers=centers,instance_index=index)
        x2, cf2, h2 = CF_Ascent(X=X,y=y,target=1.0,centers=centers,instance_index=index)

        metric_sim[0] += cf_similarity(np.array([x1]), np.array([cf1]))[0]
        metric_sim[1] += cf_similarity(np.array([x2]), np.array([cf2]))[0]

        metric_min[0] += cf_minimality(np.array([cf1]), (np.array([x1])))
        metric_min[1] += cf_minimality(np.array([cf2]), (np.array([x2])))

        metric_plaus[0] += cf_plausibility(np.array([cf1]), 1.0, X, y)
        metric_plaus[1] += cf_plausibility(np.array([cf2]), 1.0, X, y)


    metric_sim_de = metric_sim[0] / n
    metric_sim_ae = metric_sim[1] / n
    if metric_sim_de < metric_sim_ae:
        count_sim_de += 1
    else:
        count_sim_ae += 1

    metric_min_de = metric_min[0] / n
    metric_min_ae = metric_min[1] / n
    if metric_min_de > metric_min_ae:
        count_min_de += 1
    else:
        count_min_ae += 1

    metric_plaus_de = metric_plaus[0] / n
    metric_plaus_ae = metric_plaus[1] / n
    if metric_plaus_de > metric_plaus_ae:
        count_plaus_de += 1
    else:
        count_plaus_ae += 1

print("Similairty:")
print("cfde score:", count_sim_de/it)
print("cfae score:", count_sim_ae/it)
print(" ")

print("Minimality:")
print("cfde score:", count_min_de/it)
print("cfae score:", count_min_ae/it)
print(" ")

print("Plausiblity:")
print("cfde score:", count_plaus_de/it)
print("cfae score:", count_plaus_ae/it)
print(" ")

Similairty:
cfde score: 0.6
cfae score: 0.4
 
Minimality:
cfde score: 0.0
cfae score: 1.0
 
Plausiblity:
cfde score: 0.0
cfae score: 1.0
 
